In [ ]:
# Although no meaningful portion was copied directly, this work was made in collaboration with ChatGPT

In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import SimpleRNN, LSTM, Dense, Dropout
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from tensorflow.keras.models import load_model
import matplotlib.pyplot as plt
import os

2024-11-13 09:42:03.551410: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-11-13 09:42:03.555553: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-11-13 09:42:03.567842: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1731508923.588227 2367170 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1731508923.594386 2367170 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-13 09:42:03.617999: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU ins

In [2]:
all_merged_data = pd.read_csv("all_merged_data.csv")

In [3]:
sample_size = int(all_merged_data.shape[1] * 0.1)  # Calculate 10% of the columns

# Randomly select 10% of the columns
sampled_columns = all_merged_data.sample(n=sample_size, axis=1, random_state=42)  # Use random_state for reproducibility

# Separate the sampled columns as test columns and the rest as train columns
train = all_merged_data.drop(columns=sampled_columns.columns)  # Remaining 90% of columns
test = sampled_columns  # 10% of randomly selected columns
test['date'] = all_merged_data['date']  # Add 'date' back to test
train['date'] = all_merged_data['date']  # Add 'date' back to train

sample_size_dev = int(train.shape[1] * 0.1)  # Calculate 10% of the columns
# Randomly select 10% of the columns
sampled_columns_dev = train.sample(n=sample_size_dev, axis=1, random_state=42)  # Use random_state for reproducibility

# Separate the sampled columns as dev columns and the rest as true train columns
train = train.drop(columns=sampled_columns_dev.columns)  # Remaining 90% of columns
dev = sampled_columns_dev  # 10% of randomly selected columns

time_step = 50 # Define time step (number of previous observations


In [4]:
# Function to create sequences for one keyword trend
def create_sequences_for_keyword(data_column, time_step):
    X, y = [], []
    for i in range(len(data_column) - time_step):
        X.append(data_column[i:i + time_step])  # Sequence of past 'time_step' values
        y.append(data_column[i + time_step])    # The next value to predict
    return np.array(X), np.array(y)

In [ ]:
# Define the LSTM model 
model = Sequential()
model.add(LSTM(units=100, activation='tanh', input_shape=(time_step, 1), return_sequences=True))
model.add(Dropout(0.3))  
model.add(LSTM(units=100, activation='tanh'))
model.add(Dropout(0.3))
model.add(Dense(1))  # Output layer to predict the next value for the trend
number_epochs = 50
average_mse_per_epoch = []

# Compile the model 
model.compile(optimizer='adam', loss='mean_squared_error')

In [5]:
# loading the last saved model and starting with only 20 more epochs to go 

model = load_model('LSTM30Epochs.keras')
# Compile the model 
model.compile(optimizer='adam', loss='mean_squared_error')

number_epochs = 20 # only 20 left since training timed out and starting from this last checkpoint 

W0000 00:00:1731508946.160507 2367170 gpu_device.cc:2344] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


In [6]:
average_mse_per_epoch = []

for epoch in range(number_epochs):
    # Iterate over each keyword column, excluding 'date'
    for keyword in train.columns[1:]:  # Skip the 'date' column
        print(f"Training RNN model for keyword: {keyword}")
        
        # Extract the single keyword's trend data
        single_keyword_data = train[keyword].values  # Extract the column data as an array
        
        # Create sequences for the single keyword
        X, y = create_sequences_for_keyword(single_keyword_data, time_step)
        
        # Reshape X to match RNN input format (samples, timesteps, features)
        X = X.reshape((X.shape[0], X.shape[1], 1))  # 1 feature per keyword
        
        # Continue training the model on this keyword's data 
        history = model.fit(X, y, epochs=1, batch_size=16, verbose=1)


    # once all data has been seen in training for one full epoch evaluate MSE 
    mse_list = []
    for keyword in dev.columns:
        # Extract the single keyword's trend data
        single_keyword_data = dev[keyword].values  # Extract the column data as an array
    
        # Create sequences for the single keyword
        X_dev, y_dev = create_sequences_for_keyword(single_keyword_data, time_step)

        # Predict on the validation set
        y_pred = model.predict(X_dev)
    
        # Compute MSE for this keyword
        mse = mean_squared_error(y_dev, y_pred)
        mse_list.append(mse)

        
    average_mse = sum(mse_list) / len(mse_list)
    average_mse_per_epoch.append(average_mse)

    # Starting again from 30 epochs so...
    if epoch == 4: 
        model.save('LSTM35Epochs.keras')

    if epoch == 9: 
        model.save('LSTM40Epochs.keras')

    if epoch == 14: 
        model.save('LSTM45Epochs.keras')

    if epoch == 19: 
        model.save('LSTM50Epochs.keras')

   

    # Save latest dev scores
    results = pd.DataFrame({'Average MSE': average_mse_per_epoch})

    results.to_csv('AverageMSEFromTraining_from30.csv', index=False)
        

# save the final trained model after all keywords are trained and epochs are complete
model.save('FinalLSTM.keras')

Training RNN model for keyword: Book genres
13/13 ━━━━━━━━━━━━━━━━━━━━ 3s 26ms/step - loss: 0.0072
Training RNN model for keyword: Bestsellers
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step - loss: 0.0113
Training RNN model for keyword: Classic novels
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step - loss: 0.0094
Training RNN model for keyword: Mystery books
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step - loss: 0.0039
Training RNN model for keyword: Children's books
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step - loss: 0.0039
Training RNN model for keyword: Used books
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step - loss: 0.0029
Training RNN model for keyword: Graphic novels_x
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step - loss: 0.0055
Training RNN model for keyword: Young adult
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step - loss: 0.0042
Training RNN model for keyword: Historical fiction
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step - loss: 0.0048
Training RNN model for keyword: Romance novels_x
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step - loss: 0.0423
Training RNN model for keyword: Straw hats
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step - loss: 0.0289
Training RNN model for keyword: Wool sweaters
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step - loss: 0.0203
Training RNN model for keyword: Down jackets_y
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step - loss: 0.0279
Training RNN model for keyword: Overalls
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step - loss: 0.0463
Training RNN model for keyword: Tube tops
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step - loss: 0.0242
Training RNN model for keyword: Crop tops
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step - loss: 0.0182
Training RNN model for keyword: Cardigans
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step - loss: 0.0226
Training RNN model for keyword: Rompers
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step - loss: 0.0242
Training RNN model for keyword: Dress shoes
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step - loss: 0.0589
Training RNN model for keyword: Flats
13/13 ━━━━━━━━━━━━━━━━━━━

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step - loss: 0.0057
Training RNN model for keyword: Microphone boom
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step - loss: 0.0076
Training RNN model for keyword: Mini PC
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step - loss: 0.0040
Training RNN model for keyword: Slim monitor
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step - loss: 0.0060
Training RNN model for keyword: Compact case
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step - loss: 0.0065
Training RNN model for keyword: Streamers
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step - loss: 0.0073
Training RNN model for keyword: Party hats
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step - loss: 0.0054
Training RNN model for keyword: Birthday banners
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step - loss: 0.0065
Training RNN model for keyword: Confetti
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step - loss: 0.0070
Training RNN model for keyword: Table covers_x
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step - loss: 0.0061
Training RNN model for keyword: Cake toppers_x
13/

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)

